In [ ]:
%%html
  <!DOCTYPE html>
  <html lang="en">
  <head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Test</title>
    <script type="module" src="https://cdn.jsdelivr.net/npm/@ionic/core/dist/ionic/ionic.esm.js"></script>
    <script nomodule src="https://cdn.jsdelivr.net/npm/@ionic/core/dist/ionic/ionic.js"></script>
    <link rel="stylesheet" href="https://cdn.jsdelivr.net/npm/@ionic/core/css/ionic.bundle.css" />
  </head>
  <body>
    <ion-app>
      <ion-header translucent > <!-- class="ion-no-border" -->
        <ion-toolbar>
          <ion-title>CryptoVoxels Example Report!</ion-title>
        </ion-toolbar> 
      </ion-header>,
      <ion-content fullscreen class="ion-padding"> 
        <!-- 
          # https://github.com/ionic-team/ionic-docs/blob/legacy/src/demos/api/content/index.html 
          # https://github.com/ionic-team/ionic-docs/blob/legacy/src/demos/api/infinite-scroll/index.html
          # https://github.com/ionic-team/ionic-docs/blob/legacy/src/demos/api/back-button/index.html
        -->

: 

In [ ]:
%%html
<style>
  /*
  :root {
    --ion-safe-area-top: 2px;
    --ion-safe-area-bottom: 22px;
  }
  details{padding-bottom:8px}
  summary>*{ display:inline-block }
  h2, h3 {text-align: center; }
  details > summary > h3 {display: inline;}
  summary::marker { content: "🧡 "; }
  */
  h2, h3 {
    display: inline-block;
  }
  body { text-align: center; } 
  
  ion-content > div { margin-left: 25%; padding-top: 40px;  padding-bottom: 40px; }

</style> 

In [ ]:
#hide 
import sys
sys.path.append('/workspaces/3Diot.github.io/blog/')
sys.path.append('/workspaces/3Diot.github.io/docs/datasets/')